<a href="https://colab.research.google.com/github/MykhailoFokin/MachineLearning/blob/master/Naive_Bayes_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Import block
import nltk
from collections import Counter

# Define block
class NaiveBayes :

  def fit(self, input_data) :
    self.classCount = len(input_data)
    self.CountByClass = Counter(tok['KeyName'] for tok in tokens)

    

In [51]:
from collections import Counter
import itertools
# Train data
data = [["Chinese Beijing Chinese","0"],
        ["Chinese Chinese Shanghai","0"], 
        ["Chinese Macao","0"],
        ["Tokyo Japan Chinese","1"]]

# print [(k, ' '.join(v)) for k, v in data.items()]

# print(list(itertools.chain.from_iterable(data)))
# ['Chinese Beijing Chinese', '0', 'Chinese Chinese Shanghai', '0', 'Chinese Macao', '0', 'Tokyo Japan Chinese', '1']

#itemDict = {item[1]: {} for item in data}

#print(itemDict)

import nltk
class_count = {}
# nltk.download("popular")
# main_dict = dict()
dd = {item[1]: {} for item in data}
tokenized_words = []
for line in data:
  if line[1] in class_count:
    class_count[line[1]] += 1
  else :
    class_count[line[1]] = 1
  tokenized_word=nltk.word_tokenize(line[0])
  #print(tokenized_word)
  # main_dict("class":line[1] : tokenized_word)
  fdist = nltk.FreqDist(tokenized_word)
  #fdist = nltk.FreqDist(w.lower() for w in tokenized_word)
  for word in fdist:
    if word in dd[line[1]]:
      dd[line[1]][word] += fdist[word]
    else:
      dd[line[1]][word] = fdist[word]
  tokenized_words.append((dict(fdist), {"class" : line[1]}))

# from nltk.probability import FreqDist
#fdist = nltk.FreqDist(tokenized_word)
#print(fdist)


#print(tokenized_words)

#fdist.most_common(2)

print(dd)
print(class_count)

new = ["Chinese Chinese Chinese Tokyo Japan"]

tokenized_word=nltk.word_tokenize(new[0])
fdist = nltk.FreqDist(tokenized_word)
x = 0
for key in dd:
   print(key)
   for word in fdist:
     if word in dd[key]:
       y=

{'0': {'Chinese': 5, 'Beijing': 1, 'Shanghai': 1, 'Macao': 1}, '1': {'Tokyo': 1, 'Japan': 1, 'Chinese': 1}}
{'0': 3, '1': 1}
0
1
0
1
0
1
